In [7]:
# Import necessary libraries
import os
import numpy as np
import chromadb
import asyncio
import gc
import tiktoken
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain.load import dumps, loads
from langchain.schema import Document
from operator import itemgetter
from typing import Literal, List, Tuple

In [8]:
# Load API Keys from environment variables
load_dotenv()  # Load environment variables from a .env file

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")  # This key is loaded but not used in the code

# Initialize the chat model and embedding model
# ChatOpenAI is used to interact with the OpenAI GPT model, and OpenAIEmbeddings is used for generating embeddings for documents
model = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [9]:
# Özetleme zincirini oluşturma
def create_summary_chain():
    prompt_template = ChatPromptTemplate.from_template(
        "Summarize the following document in a way that best describes it semantically, considering the question and answer pairs it contains:\n\n{doc}"
    )
    chain = prompt_template | model | StrOutputParser()
    return chain

def summarize_document_with_chain(doc_content):
    """Verilen belge içeriğini LangChain ile özetler."""
    chain = create_summary_chain()
    # Belgeyi özetlemek için chain'i çalıştır
    summary = chain.invoke({"doc": doc_content})
    return summary

# Ana klasörün yolu (data klasörünün yolu)
data_directory = 'data'

# data klasörü altındaki her bir ana klasör için işlem yapıyoruz
for root, dirs, files in os.walk(data_directory):
    # root sadece bir üst seviyedeki dizini verir, bu yüzden sadece root içindeki ana klasörlerde işlemi yaparız
    if root == data_directory:
        for folder in dirs:
            folder_path = os.path.join(data_directory, folder)
            
            # _summary.txt dosyasını bu ana klasör içinde oluşturuyoruz (alfabetik olarak en üstte olacak şekilde)
            summary_file_path = os.path.join(folder_path, '_summary.txt')
            
            with open(summary_file_path, 'w') as summary_file:
                # Bu klasörün altındaki tüm dosyaları listelemek için tekrar os.walk kullanıyoruz
                for sub_root, sub_dirs, sub_files in os.walk(folder_path):
                    for file_name in sub_files:
                        if file_name != '_summary.txt' and file_name.endswith('.txt'):
                            # Her txt dosyasının tam yolunu alıyoruz
                            file_path = os.path.join(sub_root, file_name)
                            
                            # Dosyanın içeriğini okuyoruz
                            with open(file_path, 'r', encoding='utf-8') as txt_file:
                                content = txt_file.read()
                            
                            # LangChain chain yapısını kullanarak belgeyi özetliyoruz
                            summary = summarize_document_with_chain(content)
                            
                            # Dosya yolunu ve özetini _summary.txt dosyasına yazıyoruz
                            summary_file.write(f"\n=== Chunk ===\n[File path: {file_path}\nFile summary: {summary}]\n")
            
            print(f"{folder} klasörüne _summary.txt dosyası yazıldı.")

MagentaEINS klasörüne _summary.txt dosyası yazıldı.
Hilfe bei Störungen klasörüne _summary.txt dosyası yazıldı.
Geräte & Zubehör klasörüne _summary.txt dosyası yazıldı.
TV klasörüne _summary.txt dosyası yazıldı.
Vertrag & Rechnung klasörüne _summary.txt dosyası yazıldı.
Internet & Telefonie klasörüne _summary.txt dosyası yazıldı.
Apps & Dienste klasörüne _summary.txt dosyası yazıldı.
Mobilfunk klasörüne _summary.txt dosyası yazıldı.
